In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import timeit
import time
from torch.optim import Adam
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy.stats import loguniform
from sklearn.svm import SVR
import pandas as pd
import timeit
import time
from joblib import dump, load
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random

2022-03-09 19:08:22.886819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 19:08:22.886837: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
def sort(data):
    models = []
    model = np.unique(data[:,[0,1,2]], axis=0)
    
    for i in model:
        models.append(data[np.where((data[:,[0,1,2]] == i).all(axis=1))])

    return models 

def compare(model):
    rbf_results_all = []
    rbf_results_sep = []
    nn_results_all = []
    nn_results_sep = []
    
    rbf_result = np.zeros((len(model),1))
    nn_result = np.zeros((len(model),1))
    model = np.delete(model, [2,4], 1)

    rbf_result[:,0] = rbf_reg_all.predict(model[:,:-1])
    rbf_results_all.append(rbf_result)
    nn_result[:,0] = np.squeeze(nn_reg_all.predict(model[:,:-1]), axis=-1)
    nn_results_all.append(nn_result)

    # a/c separated
    rbf_result = np.zeros((len(model),1))
    nn_result = np.zeros((len(model),1))
    if model[0,0] <= 1:
        rbf_result[:,0] = rbf_reg_le.predict(model[:,:-1])
        nn_result[:,0] = np.squeeze(nn_reg_le.predict(model[:,:-1]), axis=-1)

    elif model[0,0] > 1:
        rbf_result[:,0] = rbf_reg_g.predict(model[:,:-1])
        nn_result[:,0] = np.squeeze(nn_reg_g.predict(model[:,:-1]), axis=-1)

    else:
        print("Something wrong!")

    rbf_results_sep.append(rbf_result)
    nn_results_sep.append(nn_result)
        
    return rbf_results_all, rbf_results_sep, nn_results_all, nn_results_sep



def plot(model_num):
    rbf_results_all, rbf_results_sep, nn_results_all, nn_results_sep = compare(models[model_num])
    plt.scatter(models[model_num][:,-3], models[model_num][:,-1], label="Ground Truth")
    plt.plot(models[model_num][:,-3], nn_results_all[0][:,0], label="NN all", color='red')
    plt.plot(models[model_num][:,-3], nn_results_sep[0][:,0], label="NN separated", color='green')
    plt.plot(models[model_num][:,-3], rbf_results_all[0][:,0], label="RBF all", color='black')
    plt.plot(models[model_num][:,-3], rbf_results_sep[0][:,0], label="RBF separated", color='cyan')
    plt.title("a/c={}; a/t={}; c/b={}".format(models[model_num][0,0],
                                             models[model_num][0,1],
                                             models[model_num][0,2]))
    plt.xlabel("phi")
    plt.ylabel("Mg")
    plt.legend()
    plt.show()
    return

In [10]:
# Loading test dataset
df = pd.read_csv("1_RN_data.csv")
d = df.to_numpy()
models = sort(d)
len(models)

32

In [11]:
rbf_reg_all = load("models/rbf_all.joblib")
nn_reg_all = tf.keras.models.load_model('models/NN_all.h5')

rbf_reg_le = load("models/rbf_le.joblib")
nn_reg_le = tf.keras.models.load_model('models/NN_le.h5')

rbf_reg_g = load("models/rbf_g.joblib")
nn_reg_g = tf.keras.models.load_model('models/NN_g.h5')

In [12]:
interact(plot, model_num=(1,len(models)-1,1));

interactive(children=(IntSlider(value=16, description='model_num', max=31, min=1), Output()), _dom_classes=('w…